In [1]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
# a_pattern = 'Si/SI2018/SI11318.Chn'
# 2022-nov-16: outros espectros:
# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"
# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"

# a_pattern = "Eso_non_existe.Chn"

# 2022-Dez-21 Buscando outro espectro
# a_pattern = 'CCI1603-I'
# 2022-Dez-28 Este é um espectro mais recente:
# a_pattern = 'SI09722.Chn'
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
# a_pattern = 'SI06322.Chn'
# a_pattern = 'SI06122.Chn'

In [2]:
import numpy as np
from numpy.polynomial import Polynomial as P  # 2020-09-06 Esta é a nova classe recomendada
from sklearn import linear_model

import pandas as pd
from random import randrange
# import plotly.graph_objects as go

In [3]:
from ograyspy_class import Ograyspy, select_spectrum_from_folder_list
from spec_class import Spec

In [4]:
ogra = Ograyspy()
print(ogra.info_node)
to_be_found = 'Genie_Transfer'
# to_be_found = 'some_spectra' # NAO ESTAH ACHANDO localmente no note!

Found folder name:  /home/marcelofm/snap
Parts:  ('/', 'home', 'marcelofm', 'snap')
No. spec files:  0
marcelofm-X540LA


In [5]:
print('\nExec ogra.define_files_folder(to_be_found):')
ogra.define_files_folder(to_be_found)
print(ogra.pkl_folder_files)
spectra_list_df = pd.read_pickle(ogra.pkl_folder_files)
del ogra


Exec ogra.define_files_folder(to_be_found):
Found folder name:  /home/marcelofm/Nextcloud/Genie_Transfer
Parts:  ('/', 'home', 'marcelofm', 'Nextcloud', 'Genie_Transfer')
No. spec files:  7292
Linuxmarcelofm-X540LA.pkl


In [6]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [7]:
len(reduc_nms)

7292

In [8]:
# 2022-Dez-28 Esse é um espectro mais recente que posso comparar com saída do Genie:
a_pattern = 'SI06122.Chn'

In [9]:
f_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)
f_name, reduced_f_name

Existing:
Achou! indice=904, nomearq = Si/SI2022/SI06122.Chn
Final choices:
spectra_path: /home/marcelofm/Nextcloud/Genie_Transfer
a_spec_name: /home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2022/SI06122.Chn
reduced_f_name: Si/SI2022/SI06122.Chn


(PosixPath('/home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2022/SI06122.Chn'),
 'Si/SI2022/SI06122.Chn')

In [10]:
nucl_iear1_df = pd.read_pickle('nucl_iear1_list.pkl')
nucl_iear1_df

,nuclide_name,energy,unc_en,intensity,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma
1,24na,1368.6250,0.005,99.994,0.002,14.956,B-,100.0,1368.6,True
2,24na,2754.0080,0.011,99.867,0.01,14.956,B-,100.0,1368.6,False
9,51cr,320.0824,0.0004,9.910,0.01,27.704,EC,100.0,320.1,True
15,54mn,834.8480,0.003,99.976,0.001,312.200,EC+B+,100.0,834.8,True
22,58co,810.7593,0.002,99.450,0.01,70.860,EC+B+,100.0,810.8,True
...,...,...,...,...,...,...,...,...,...,...
1066,228Ac,89.9540,,2.010,0.19,6.150,B-,100.0,911.6,False
1067,228Ac,93.3470,,3.200,0.3,6.150,B-,100.0,911.6,False
1068,228Ac,105.5660,,1.160,0.11,6.150,B-,100.0,911.6,False
1069,228Ac,106.8940,,1.540,0.15,6.150,B-,100.0,911.6,False


In [11]:
# 2023-Jun-15: Setting gamma lines/ranges to dismiss in the analysis
df1 = nucl_iear1_df
df1["is_to_consider"] = True
df1.loc[((df1.energy > 509) & (df1.energy < 513)) | (df1.energy < 100), "is_to_consider"] = False
# Convert these columns from object to numeric; empty strings turns into NaNs
df1["unc_i"] = pd.to_numeric(df1.unc_i, errors="coerce")
df1["unc_en"] = pd.to_numeric(df1.unc_en, errors="coerce")
del df1
nucl_iear1_df

,nuclide_name,energy,unc_en,intensity,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider
1,24na,1368.6250,0.0050,99.994,0.002,14.956,B-,100.0,1368.6,True,True
2,24na,2754.0080,0.0110,99.867,0.010,14.956,B-,100.0,1368.6,False,True
9,51cr,320.0824,0.0004,9.910,0.010,27.704,EC,100.0,320.1,True,True
15,54mn,834.8480,0.0030,99.976,0.001,312.200,EC+B+,100.0,834.8,True,True
22,58co,810.7593,0.0020,99.450,0.010,70.860,EC+B+,100.0,810.8,True,True
...,...,...,...,...,...,...,...,...,...,...,...
1066,228Ac,89.9540,NaN,2.010,0.190,6.150,B-,100.0,911.6,False,False
1067,228Ac,93.3470,NaN,3.200,0.300,6.150,B-,100.0,911.6,False,False
1068,228Ac,105.5660,NaN,1.160,0.110,6.150,B-,100.0,911.6,False,True
1069,228Ac,106.8940,NaN,1.540,0.150,6.150,B-,100.0,911.6,False,True


In [12]:
a_spec = Spec(f_name, reduced_f_name)

In [13]:
dir(a_spec)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_engy_to_pks_df',
 'channel_energy_calib',
 'create_activities_df',
 'create_matching_peaks_df',
 'curr_h_win',
 'det_descr',
 'en_eff',
 'f_name',
 'final_composed_baseline',
 'generate_pandas_dataframe',
 'identify_nuclides',
 'lv_time',
 'net_spec_ser_an',
 'nucl_an',
 'origin_spec_ser_an',
 'read_pkl_file',
 'reduced_f_name',
 'results_pkl_file',
 'rl_time',
 'sam_descr',
 'source_datetime',
 'spec_io',
 'start_datetime',
 'sufx',
 'total_analysis']

In [14]:
a_spec.total_analysis(gener_dataframe=True, results_path='../ograyspy_results')
# A opção acima, gener_dataframe=True,  grava o pkl_file do espectro
vars(a_spec)

Starting Spec.total_analysis...
resolve_peaks_and_regions:
define_multiplets_regions completado. Define: self.mix_regions.
resolve_peaks_and_regions:
define_multiplets_regions completado. Define: self.mix_regions.
Si/SI2022/SI06122.Chn
Si/SI2022/SI06122_result.pkl
results_pkl_file
../ograyspy_results/Si/SI2022/SI06122_result.pkl
Finish Spec.total_analysis!


{'net_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x7fd06c7d9bd0>,
 'final_composed_baseline': <generic_series_analysis_class.GenericSeriesAnalysis at 0x7fd06c504cd0>,
 'f_name': PosixPath('/home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2022/SI06122.Chn'),
 'reduced_f_name': 'Si/SI2022/SI06122.Chn',
 'sufx': '.chn',
 'spec_io': None,
 'lv_time': 50000.0,
 'rl_time': 50028.96,
 'source_datetime': None,
 'results_pkl_file': PosixPath('../ograyspy_results/Si/SI2022/SI06122_result.pkl'),
 'origin_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x7fd0b05059d0>,
 'start_datetime': datetime.datetime(2022, 7, 27, 16, 56, 29),
 'det_descr': 'MAQUINA_01 MCB 1 Input 1',
 'sam_descr': 'SI-061/22 Agua da Piscina Reator IEA-R1 (GMX)',
 'channel_energy_calib': <genericcalib_class.ChannelEnergyCalib at 0x7fd06ebf7810>,
 'nucl_an': <nuclide_analysis_class.NuclideAnalysis at 0x7fd06d2d77d0>}

In [15]:
# 2023-Oct-18 PAREI AQUI. Reformatar-deletar tudo o que segue.

In [16]:
spectra_list_df

,info_plat,info_mach,info_syst,info_node,home_path,spectra_path,n_files,a_spec_ind,a_spec_name,reduced_f_name,files_list,reduced_names_files_list,spectra_pattern_names,pkl_folder_files,results_path_name
0,Linux-6.2.0-35-generic-x86_64-with-glibc2.37,x86_64,Linux,marcelofm-X540LA,/home/marcelofm,/home/marcelofm/Nextcloud/Genie_Transfer,7292,0,,,[/home/marcelofm/Nextcloud/Genie_Transfer/BGNO...,"[BGNOVO.Chn, testes_lowbg_planar/Ba133 Planar....","[**/*.[Cc][Hh][Nn], **/*.[Ii][Ee][Cc]]",Linuxmarcelofm-X540LA.pkl,../ograyspy_results


In [17]:
nucl_iear1_selctd_gamms_df = nucl_iear1_df.loc [
    (nucl_iear1_df.intensity > 1.0) & nucl_iear1_df.is_to_consider
]
nucl_iear1_selctd_gamms_df

,nuclide_name,energy,unc_en,intensity,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider
1,24na,1368.6250,0.0050,99.994,0.002,14.956,B-,100.0,1368.6,True,True
2,24na,2754.0080,0.0110,99.867,0.010,14.956,B-,100.0,1368.6,False,True
9,51cr,320.0824,0.0004,9.910,0.010,27.704,EC,100.0,320.1,True,True
15,54mn,834.8480,0.0030,99.976,0.001,312.200,EC+B+,100.0,834.8,True,True
22,58co,810.7593,0.0020,99.450,0.010,70.860,EC+B+,100.0,810.8,True,True
...,...,...,...,...,...,...,...,...,...,...,...
944,228Ac,968.9710,0.0170,15.800,0.300,6.150,B-,100.0,911.6,False,True
1014,228Ac,1588.2000,0.0300,3.220,0.080,6.150,B-,100.0,911.6,False,True
1017,228Ac,1630.6270,0.0100,1.510,0.040,6.150,B-,100.0,911.6,False,True
1068,228Ac,105.5660,NaN,1.160,0.110,6.150,B-,100.0,911.6,False,True


In [18]:
nucl_iear1_selctd_gamms_df.dtypes

nuclide_name       object
energy            float64
unc_en            float64
intensity         float64
unc_i             float64
half_life         float64
decay              object
decay_%           float64
key_gamma         float64
is_key_gamma       object
is_to_consider       bool
dtype: object

In [19]:
# Primeiro fiz assim:
# cross_df = pd.merge(pks_comprehensive_df, nucl_iear1_selctd_gamms_df, how='cross')
# cross_df["delta_en"] = cross_df.engy_pk_det - cross_df.energy
# cross_df

# Agora fazer de novo:
# cross_df_2 = pd.merge(matching_peaks_df, nucl_iear1_selctd_gamms_df, how='cross', suffixes=('_x', None))
# cross_df_2.drop(cross_df_2.filter(regex='_x$').columns, axis=1, inplace=True)
# cross_df_2["delta_en"] = cross_df_2.recalib_engy_ransac - cross_df_2.energy
# cross_df_2

In [20]:
# cross_df = pd.merge(pks_comprehensive_df, nucl_iear1_selctd_gamms_df, how='cross')
 #    cross_df["delta_en"] = cross_df.engy_pk_det - cross_df.energy
  #  cross_df 

In [21]:
dir(a_spec)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_engy_to_pks_df',
 'channel_energy_calib',
 'create_activities_df',
 'create_matching_peaks_df',
 'curr_h_win',
 'det_descr',
 'en_eff',
 'f_name',
 'final_composed_baseline',
 'generate_pandas_dataframe',
 'identify_nuclides',
 'lv_time',
 'net_spec_ser_an',
 'nucl_an',
 'origin_spec_ser_an',
 'read_pkl_file',
 'reduced_f_name',
 'results_pkl_file',
 'rl_time',
 'sam_descr',
 'source_datetime',
 'spec_io',
 'start_datetime',
 'sufx',
 'total_analysis']

In [22]:
a_spec.results_pkl_file

PosixPath('../ograyspy_results/Si/SI2022/SI06122_result.pkl')

In [23]:
vars(a_spec)

{'net_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x7fd06c7d9bd0>,
 'final_composed_baseline': <generic_series_analysis_class.GenericSeriesAnalysis at 0x7fd06c504cd0>,
 'f_name': PosixPath('/home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2022/SI06122.Chn'),
 'reduced_f_name': 'Si/SI2022/SI06122.Chn',
 'sufx': '.chn',
 'spec_io': None,
 'lv_time': 50000.0,
 'rl_time': 50028.96,
 'source_datetime': None,
 'results_pkl_file': PosixPath('../ograyspy_results/Si/SI2022/SI06122_result.pkl'),
 'origin_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x7fd0b05059d0>,
 'start_datetime': datetime.datetime(2022, 7, 27, 16, 56, 29),
 'det_descr': 'MAQUINA_01 MCB 1 Input 1',
 'sam_descr': 'SI-061/22 Agua da Piscina Reator IEA-R1 (GMX)',
 'channel_energy_calib': <genericcalib_class.ChannelEnergyCalib at 0x7fd06ebf7810>,
 'nucl_an': <nuclide_analysis_class.NuclideAnalysis at 0x7fd06d2d77d0>}

In [23]:
/home/marcelofm/ograyspy_results/Si/SI2022

In [23]:
new_ch_en_calib

In [24]:
def en_eff(engy):
    return np.exp(p_eff(np.log(engy)))

'SI06122_result_activ_list'

In [25]:
en_eff(1332)

../ograyspy_results/Si/SI2022/SI06122_result.pkl


Lets identify nuclides.
    peaks   fwhm_centr   rough_sums    centroids     variances       pk_hei  \
0      30    29.785567  1893.942368    29.963695  14133.214332   818.646195   
1     128   128.364887  2876.788495   127.767831  11283.211505   720.585627   
2     145   144.459911  1278.607977   144.896569  10161.329960   389.571554   
3     236   235.273124   462.759320   235.202199  12019.150830   123.425784   
4     396   396.401754   630.310441   396.426778  10404.641509   162.845302   
5     489   488.997063   249.976688   488.967375   4072.023312   125.089174   
6     581   581.463638   430.396173   581.647407   5405.821287   147.276132   
7     716   715.508125   222.738094   715.666805   4033.829966    79.006167   
8     841   841.967813  2900.206925   841.976062   7873.573309   559.895371   
9     960   960.088168   483.773366   960.065118   3335.841582   148.624963   
10   1003  1002.888165   769.886444  1002.931019   3458.113556   220.739274   
11   1011  1011.595555   246

In [27]:
# 2023-Sep-21
# Now use the assembled peak df, read_result_df, to proceed.

final_pkl_file:
../ograyspy_results/Si/SI2022/SI06122_final.pkl


In [28]:
seconds = a_spec.results_pkl_file # .with_stem('second_result')
seconds
activ_stem = a_spec.results_pkl_file.stem + '_activ_list'
activ_stem

'SI06122_result_activ_list'

../ograyspy_results/Si/SI2022/SI06122_result.pkl


Lets identify nuclides.
    peaks   fwhm_centr   rough_sums    centroids     variances       pk_hei  \
0      30    29.785567  1893.942368    29.963695  14133.214332   818.646195   
1     128   128.364887  2876.788495   127.767831  11283.211505   720.585627   
2     145   144.459911  1278.607977   144.896569  10161.329960   389.571554   
3     236   235.273124   462.759320   235.202199  12019.150830   123.425784   
4     396   396.401754   630.310441   396.426778  10404.641509   162.845302   
5     489   488.997063   249.976688   488.967375   4072.023312   125.089174   
6     581   581.463638   430.396173   581.647407   5405.821287   147.276132   
7     716   715.508125   222.738094   715.666805   4033.829966    79.006167   
8     841   841.967813  2900.206925   841.976062   7873.573309   559.895371   
9     960   960.088168   483.773366   960.065118   3335.841582   148.624963   
10   1003  1002.888165   769.886444  1002.931019   3458.113556   220.739274   
11   1011  1011.595555   246

In [ ]:
# 2023-Oct-24: FIX
# Integrate what follows here

In [8]:
analysis_df.reset_index(drop=True, inplace=True)

In [9]:
analysis_df['test'] = analysis_df['decay_%']*analysis_df['intensity']/100

In [12]:
analysis_df

,nuclide_name,centroids,rough_sums,variances,recalib_energy,energy,intensity,unc_i,decay_%,is_key_gamma,nuclide_name,disintegr,activity_conc,test
0,212Pb,396.426778,630.310441,10404.641509,238.770932,238.6320,43.60000,0.5307178534777213,100.000,True,212Pb,7.324243e+04,14.648486,43.600000
1,214Pb,488.967375,249.976688,4072.023312,295.305981,295.2240,18.47088,0.11249527437185973,100.000,False,214Pb,8.502770e+04,17.005540,18.470880
2,214Pb,581.647407,430.396173,5405.821287,351.925344,351.9320,35.72370,0.2352736185805795,100.000,True,214Pb,8.929519e+04,17.859037,35.723700
3,208Tl,960.065118,483.773366,3335.841582,583.096300,583.1870,85.00000,0.3,100.000,True,208Tl,6.280187e+04,12.560374,85.000000
4,214Bi,1002.931019,769.886444,3458.113556,609.281662,609.3210,45.44000,0.19,99.979,True,214Bi,1.925320e+05,38.506404,45.430458
5,110ag,1082.200436,2125.483990,4570.516010,657.704238,657.7600,95.61000,0.09,98.670,True,110ag,2.655030e+05,53.100602,94.338387
6,137cs,1088.105023,401.081225,2556.918775,661.311093,661.6570,85.10000,0.2,100.000,True,137cs,5.650064e+04,11.300127,85.100000
7,110ag,1115.022855,176.695976,1999.057501,677.753980,677.6217,10.70000,0.05,98.670,False,110ag,2.009797e+05,40.195943,10.557690
8,110ag,1161.943753,336.295814,2591.560129,706.415658,706.6760,16.69000,0.07,98.670,False,110ag,2.517501e+05,50.350025,16.468023
9,131i,1189.208186,160.747926,1865.322724,723.070063,722.9110,1.76855,0.026749,100.000,False,131i,1.151661e+06,230.332119,1.768550


In [18]:
analysis_df.reindex(['nuclide_name'])

,nuclide_name,centroids,rough_sums,variances,recalib_energy,energy,intensity,unc_i,decay_%,is_key_gamma,nuclide_name,disintegr,activity_conc,test
nuclide_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# d5_where = d5.where(d5.nuclide_name=="110ag")
# d5_where
d5_loc = analysis_df.loc[analysis_df.nuclide_name=="110ag"]
d5_loc

ValueError: Index data must be 1-dimensional

In [14]:
# 2023-Ago-4 PAREI AQUI. Fazer a indexação para o cálculo da atividade considerando a média das linhas de cada nuclídeo
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#dataframe-column-selection-in-groupby
d5_grouped = analysis_df.groupby("nuclide_name")[['energy','intensity','centroids','rough_sums','variances','disintegr']]
# d5_grouped = d5.groupby("nuclide_name")[['disintegr']]
# sdfsdf = pd.DataFrame([[d5_grouped.mean(), d5_grouped.median()]]) 
# sdfsdf


ValueError: Grouper for 'nuclide_name' not 1-dimensional

In [ ]:
d5_grouped.mean()

In [64]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#built-in-aggregation-methods
d5_aggr_df = pd.DataFrame(df_result.groupby("nuclide_name")[['disintegr']].mean())
d5_aggr_df['dsnt_median'] = df_result.groupby("nuclide_name")[['disintegr']].median()
d5_aggr_df['dsnt_counts'] = df_result.groupby("nuclide_name")[['disintegr']].count()
d5_aggr_df['dsnt_sem'] = df_result.groupby("nuclide_name")[['disintegr']].sem()
d5_aggr_df['dsnt_std'] = df_result.groupby("nuclide_name")[['disintegr']].std()
d5_aggr_df

ValueError: Grouper for 'nuclide_name' not 1-dimensional

In [ ]:
df_result.groupby("nuclide_name")[['disintegr']].describe()

In [ ]:
d5_aggr_df['disint_mean'] = d5_grouped.mean()
d5_aggr_df['disint_count'] = d5_grouped.count()
d5_aggr_df

In [ ]:
df_result

In [ ]:
help(spec_engy_identif)

In [ ]:
# 2023-Jun-13
# PAREI AQUI
# Trazer o trabalho feito em
# radionuclide_analysis.ipynb e adaptar ao que está aqui
# especialmente na identificaćão dos nucl[ideos

# Depois, deletar radionuclide_analysis.ipynb.

In [4]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)


df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)


df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11],
)


frames = [df1, df2, df3]

result = pd.concat(frames)
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


In [5]:
result = pd.concat(frames, keys=["x", "y", "z"])
result

A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11

In [6]:
result.loc[result.A=='A1'].loc[result.C=='C0']

,,A,B,C,D


In [7]:
# https://stackoverflow.com/questions/63529555

In [8]:
animals = ['cat', 'dog', 'hamster', 'dolphin']

In [9]:
df_animals_sets = pd.DataFrame(data={
    'id': [1,2,3,4,5],
    'animals': ['dog,cat','dog','cat,dolphin','cat,dog','hamster,dolphin']
})
df_animals_sets

,id,animals
0,1,"dog,cat"
1,2,dog
2,3,"cat,dolphin"
3,4,"cat,dog"
4,5,"hamster,dolphin"


In [10]:
df = df_animals_sets
df_final = (df.astype(str).assign(animals=df.animals.str.split(','))
                          .explode('animals').groupby('animals').id.agg(','.join)
                          .reset_index())
df_final

,animals,id
0,cat,"1,3,4"
1,dog,"1,2,4"
2,dolphin,"3,5"
3,hamster,5


In [1]:
# 2023-Oct-24 FIX:
# Brought from extinct radionuculde_analysis.ipynb
# Jupyter notebook for tests with OGRaySpY modules

In [2]:
pip list

Package                       Version
----------------------------- --------
alabaster                     0.7.13
anyio                         3.7.0
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
asttokens                     2.2.1
attrs                         23.1.0
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.12.2
bleach                        6.0.0
certifi                       2023.5.7
cffi                          1.15.1
charset-normalizer            3.1.0
comm                          0.1.3
debugpy                       1.6.7
decorator                     5.1.1
defusedxml                    0.7.1
docutils                      0.19
executing                     1.2.0
fastjsonschema                2.17.1
fqdn                          1.5.1
idna                          3.4
imagesize                     1.4.1
ipykernel                     6.23.1
ipython       

In [3]:
import numpy as np
from numpy.random import default_rng
from numpy.polynomial import Polynomial as P
from scipy import special
import pandas as pd

from ograyspy_class import select_spectrum_from_folder_list
from spec_class import Spec

In [4]:
# https://lmfit.github.io/lmfit-py/builtin_models.html
from lmfit import Model, minimize, Parameters, report_fit, fit_report, printfuncs
from lmfit.models import LinearModel, StepModel, ExponentialModel, LorentzianModel
from lmfit.models import GaussianModel, SkewedGaussianModel, ExpressionModel

ModuleNotFoundError: No module named 'lmfit'

In [ ]:
from ograyspy_class import Ograyspy

In [ ]:
ogra = Ograyspy(batch_mode=False, folder_to_find='Genie_Transfer')
print('Objeto ogra: Ograyspy criado.')
print(f'ogra.info_plat: {ogra.info_plat}')
print(f'ogra.info_mach: {ogra.info_mach}')
print(f'ogra.info_syst: {ogra.info_syst}')
print(f'ogra.info_node: {ogra.info_node}')
print(f'ogra.home_path: {ogra.home_path}')

In [ ]:
# a spectrum pattern to find and test:

In [ ]:
# 2022
# Se quiser ignorar o espectro aleatório, definir aqui um específico. Se não, pular.
# Vou ignorar o espectro já analisado e refazer a análise, para testes.
# 2022-Mar-24 Resolver essa questão dos nomes de arquivos em Linux e Windows
# a_spec_name = '..\\..\\Genie_Transfer\\Si\\2018\\PNR\\PRN0803.Chn'
# a_spec_name = '../../Genie_Transfer/Si/2018/PNR/PRN0803.Chn'
# a_spec_name = '../../Genie_Transfer/Si/2018/CTP/CTP0908-2.Chn'
# Esse é um que dá problema na baseline perto do 159 keV do I-123:
# a_spec_name = '../../Genie_Transfer/Si/2018/CCI/CCI1508I.Chn'
# Precisa quebrar mais a baseline: Ok, consertado!!!
# 2022: Já usando o GMX:
# a_spec_name = '../../Genie_Transfer/Filtros/2022/Cci/CCI1603-I.Chn'



In [ ]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
# a_pattern = 'Si/SI2018/SI11318.Chn'
# 2022-nov-16: outros espectros:
# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"
# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"

# a_pattern = "Eso_non_existe.Chn"

# 2022-Dez-21 Buscando outro espectro
# a_pattern = 'CCI1603-I'
# 2022-Dez-28 Este é um espectro mais recente:
# a_pattern = 'SI09722.Chn'
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
a_pattern = 'SI06322.Chn'

In [ ]:
dir(ogra)

In [ ]:
vars(ogra)

In [ ]:
# Select a machine/system to read from:
spectra_list_df = pd.read_pickle(ogra.pkl_folder_files)
spectra_list_df

In [ ]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [ ]:
reduc_nms

In [ ]:
a_spec_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)

In [ ]:
a_spec_name, reduced_f_name

In [ ]:
a_spec = Spec(a_spec_name, reduced_f_name)

In [ ]:
vars(a_spec)

In [ ]:
# 2023-Jun-13
# PAREI AQUI: levar tudo que está aqui para
# nuclide_identification.ipynb


In [ ]:
p_en = a_spec.channel_energy_calib.p_en
p_en

In [ ]:
peaks_df['energy'] = p_en(peaks_df['centroids'])
peaks_df

In [ ]:
# 2023-Mar-21: Identify nuclide candidates in peaks_df from pisciro_eng_df, with energy tolerance en_toler

In [ ]:
big_merge = pd.merge(peaks_df, pisciro_eng_df, how='cross')
big_merge

In [ ]:
en_toler = 2.0

In [ ]:
big_merge['delta_en'] = big_merge.energy - big_merge.Engy
big_merge

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#selection-by-callable
# MaTched ENergies
mt_en = pd.DataFrame(big_merge.loc[lambda df: abs(df['delta_en']) < en_toler, :]) \
[['Engy', 'Nucl', 'rough_sums', 'Yie']]
mt_en
# mt_en[['Engy', 'Nucl']]

In [ ]:

msedg serg set asrhg saertrg

In [ ]:
rng = np.random.default_rng()
# rints = rng.integers(low=0, high=10, size=3)

# 2022-Jun-23: Reativar após consertar o arquivo pickle:
# rints = rng.integers(pkl_df1.count()[0], size=3)
# print(rints)
# type(rints[0])
# some_spec = rints[1]
# pkl_df1.iloc[some_spec]
# a_spec_name = pkl_df1.iloc[some_spec]['FName']
# a_spec_name

In [ ]:
# 2022-Dez-28: Atualizar:
# (2022-Jun-24 Lembrando como estão os defaults de total_analysis(...):
# def total_analysis(self, k_sep_pk=2.0, smoo=3000.0, widths_range=(4.0, 20.0)):)
# smoo=3000 fica razoável, mas faz umas ondas
# smoo=1500 fica horrível, faz uns abismos. Quanto menor, pior
# smoo=400 fica razoável, mas faz umas ondas
# ogra.perform_total_analysis(
#     k_sep_pk=2.0, smoo=4096, widths_range=(4.0, 20.0)
# )

In [ ]:
# 2022-Jun-14
# Verificando o que se tem até agora em a_raw_spec:
# (Pode pular e ir direto aos gráficos)

In [ ]:

# Testes:
# https://numpy.org/doc/stable/reference/random/generator.html#distributions
# x = np.linspace(0, 10, N) + rng.normal(loc=7, scale=0.05, size=N)
# y = np.linspace(22, 23, N)

# Busca do canal onde a altura do espectro se "estabiliza"
# https://numpy.org/doc/stable/reference/routines.sort.html#searching



In [ ]:
vars(pkprms)

In [ ]:
# 2022-Dez-21 Achar o pico de maior área inicial do espectro:

In [ ]:
largest_pk = np.argmax(pkprms.rough_sums)
pkprms.centroids[largest_pk]

In [ ]:
# Só pra lembrar como faz divisão inteira
21 // 7

In [ ]:
wi_re = pkprms.wide_regions[largest_pk]
x = np.linspace(wi_re[0], wi_re[1], num=wi_re[1]-wi_re[0]+1)
y = cntarr[wi_re[0]:wi_re[1]+1]
c1 =  pkprms.centroids[largest_pk]
s1 = pkprms.propts['widths'][largest_pk]*pkprms.k_fwhm_to_sd
a1 = pkprms.rough_sums[largest_pk]
# c2 = 313
# s2 = 1.5
# a2 = 400
sste = 2.0

In [ ]:
# 2023-Jan-3
# Aqui finalizo este teste:
# Dado um espectro, escolhi o pico mais proeminente do espectro
# e fiz o ajuste gaussianco com lmfit
# No caso, o pico tem área, centroide, SD e FWHM:
# g1_amplitude:  5264.84728 +/- 85.1465566 (1.62%) (init = 5347.497)
# g1_center:     1926.29564 +/- 0.02690572 (0.00%) (init = 1926.275)
# g1_sigma:      1.44040103 +/- 0.02690060 (1.87%) (init = 1.418733)
# g1_fwhm:       3.39188516 +/- 0.06334608 (1.87%) == '2.3548200*g1_sigma'
# g1_height:     1458.18438 +/- 23.5824677 (1.62%) == '0.3989423*g1_amplitude/max(1e-15, g1_sigma)'

# Comparando com saíuda do Genie:
# canal (>0)    En:    FWHM (keV)    net area    net area unc   continuum
# 1927.34      1173.47 2.04          5.40E+003   89.73          8.83E+002
#

In [ ]:
out

In [ ]:
out.params

In [ ]:
out.var_names

In [ ]:
out.params.items()

In [ ]:
out.params.pretty_print()

In [ ]:
print('-------------------------------')
print('Parameter    Value       Stderr')
for name, param in out.params.items():
    # print(f'{name:7s} {param.value:11.5f} {param.stderr:11.5f}')
    print(name)
    print(param)
    print(param.value)

In [ ]:
def gauss_lmfit_singlet(pkprms, ipk):
    wi_re = pkprms.wide_regions[ipk]
    print(f'pico: {wi_re}')
    x = np.linspace(wi_re[0], wi_re[1], num=wi_re[1]-wi_re[0]+1)
    y = cntarr[wi_re[0]:wi_re[1]+1]
    c1 = pkprms.centroids[ipk]
    s1 = pkprms.propts['widths'][ipk]*pkprms.k_fwhm_to_sd
    a1 = pkprms.rough_sums[ipk]

    parames = Parameters()

    gauss1 = GaussianModel(prefix='g1_')
    parames.update(gauss1.make_params())
    parames['g1_center'].set(value=c1, min=c1-2, max=c1+2)
    parames['g1_sigma'].set(value=s1, min=0.5, max=10)
    parames['g1_amplitude'].set(value=a1, min=10)

    mod = gauss1

    init = mod.eval(parames, x=x)
    out = mod.fit(y, parames, x=x, verbose=False)
    print('Fez.')

    # ret = [param for param in out.params.items()]
    ret = out
    return ret

In [ ]:
for ipk in range(len(pkprms.peaks)):
    ret = gauss_lmfit_singlet(pkprms, ipk)
    print(ret)

In [ ]:
ret

In [ ]:
ret.params

In [ ]:
for name, param in ret.params.items():
    print(name, param.value, param.stderr)

In [ ]:
for ipk in range(len(pkprms.peaks)):
    ret = gauss_lmfit_singlet(pkprms, ipk)
    # ret.params.pretty_print()

In [ ]:
ret

In [ ]:
xfgn sr hsrthf srth

In [ ]:
my_df.dtypes

In [ ]:
ch_en = ogra.a_spec.channel_energy_calib.p_en

In [ ]:
my_df['centrs_en'] = ch_en(my_df['centroids'])
my_df

In [11]:
# The End